In [129]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
import sklearn.model_selection as skm
import sklearn.linear_model as skl
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from functools import partial

In [130]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP.models import \
(Stepwise,
sklearn_selected,
sklearn_selection_path)
!pip install l0bnb
from l0bnb import fit_path

In [131]:
Hitters = load_data('Hitters')
Hitters = Hitters.dropna();

split features and outcome

In [132]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)
X = pd.DataFrame(design.transform(Hitters))
X.reset_index(inplace = True)
X.drop(['intercept','index'], axis = 'columns', inplace = True)

C:\Users\Asus\anaconda3\Lib\site-packages\ISLP\models\columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
C:\Users\Asus\anaconda3\Lib\site-packages\ISLP\models\columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
C:\Users\Asus\anaconda3\Lib\site-packages\ISLP\models\columns.py:151: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if is_categorical[i]:
C:\Users\Asus\anaconda

In [133]:
X

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League[N],Division[W],PutOuts,Assists,Errors,NewLeague[N]
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,1.0,1.0,632,43,10,1.0
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,0.0,1.0,880,82,14,0.0
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,1.0,0.0,200,11,3,1.0
3,321,87,10,39,42,30,2,396,101,12,48,46,33,1.0,0.0,805,40,4,1.0
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,0.0,1.0,282,421,25,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,497,127,7,65,48,37,5,2703,806,32,379,311,138,1.0,0.0,325,9,3,1.0
259,492,136,5,76,50,94,12,5511,1511,39,897,451,875,0.0,0.0,313,381,20,0.0
260,475,126,3,61,43,52,6,1700,433,7,217,93,146,0.0,1.0,37,113,7,0.0
261,573,144,9,85,60,78,8,3198,857,97,470,420,332,0.0,0.0,1314,131,12,0.0


In [134]:
Y = np.array(Hitters['Salary'])

In [135]:
#show all of number digits
np.set_printoptions(suppress=True)

In [136]:
def RSS(coef, intercept, X, Y):
    RSS = 0;
    for i_sample in range(np.shape(X)[0]):
        #print('Sample X: ', X.loc[i_sample].values)
        #print('Sample Y: ', Y[i_sample])
        Y_hat = (np.dot(X.loc[i_sample].values, coef) + intercept)
        RSS += (Y[i_sample] - Y_hat)**2
    return RSS

In [137]:
# X = X.head(4)
# Y = Y[0:4]
# Y

In [138]:
F = LinearRegression()
F.fit(X, Y)

LinearRegression()

In [139]:
print(f"Intercept: {F.intercept_}\n\
Coefficients: {F.coef_}")

Intercept: 163.10358775118237
Coefficients: [  -1.9798729     7.50076754    4.3308829    -2.37620998   -1.04496196
    6.23128632   -3.48905426   -0.17134047    0.13399096   -0.17286107
    1.45430494    0.8077088    -0.81157091   62.59942304 -116.84924564
    0.28189251    0.37106921   -3.36076048  -24.76232511]


In [140]:
#sigma2 = RSS(F.coef_, F.intercept_, X, Y)/(263-20)

In [141]:
sigma2

99591.35617968222

In [142]:
def FowardSelection(Y, X, k):
    selection = []
    rest = X.columns

    #i = 1
    for i in range(1, k+1):
        rss = np.inf
        sele = selection
        selection.append(None)
        for feature in rest:
            #remove unnessesary feature
            X_temp = X.filter(sele + [feature])
            
            #create linear model
            F = LinearRegression()
            F.fit(X_temp, Y)
            
            #calculate rss of model
            rss_temp = RSS(F.coef_, F.intercept_, X_temp, Y)
            
            # choose feature having minimum rss and append to selection
            if rss > rss_temp:
                rss = rss_temp
                selection.pop()
                selection.append(feature)

    return selection          

In [143]:
FowardSelection(Y,X,10)

['PutOuts',
 'Division[W]',
 'CHits',
 'CAtBat',
 'CHmRun',
 'Walks',
 'Hits',
 'HmRun',
 'Runs',
 'RBI']